# Detección de fraudes en transacciones de tarjetas de crédito 💳

## Requisitos y dependencias

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !conda install -y anaconda::scikit-learn

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

## Extracción de datos 🚀

In [ ]:
df_sub      = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/sample_submission.csv')
df_test     = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/test.csv')
df_train    = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/train.csv')

## Etapa de analisis *EDA* 👨‍💻

### Visualizo datos *Entrenamiento*

In [ ]:
df_train.info()

#### Verifico los datos que contiene mi variable objetivo

In [ ]:
df_train['IsFraud'].head(10)

In [ ]:
df_train[['IsFraud']].groupby('IsFraud').count().head()

### Separar variasbles númericas *(dimensionales)* con ordinales *(categóricas)*
- **En este caso solo tenemos**

### Correlación

In [ ]:
# Carga tu DataFrame
df = df_train

# Calcula la matriz de correlación
correlation_matrix = df.corr()

# Crea el mapa de calor
plt.figure(figsize=(30, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Mapa de calor de correlación')
plt.show()


La correlación es muy baja, no es necesario hacer pca

### Detección de datos anómalos 

In [ ]:
# Assuming your DataFrame is named 'data' and has numerical columns
data_to_plot = df_train.select_dtypes(include=[np.number])  # Select numerical columns

# Create the boxplot
plt.figure(figsize=(45, 45))  # Adjust figure size as needed
data_to_plot.boxplot()

# Customize the plot (optional)
plt.xlabel("Column Names")
plt.ylabel("Values")
plt.title("Boxplots")
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for better readability
plt.grid(True)

# Display the plot
plt.tight_layout()
plt.show()


Lo que quiere decir que mis datos no contienen outlayers

## Etapa de limpieza

In [ ]:
df_train = df_train.drop(columns=['id'])
df_test = df_test.drop(columns=['id'])
var_obj = df_train['IsFraud']
df_train = df_train.drop(columns=['IsFraud'])

## Etapa de Pre-proceso de datos 🧠
**Notas**: 
- la variable objetivo *IsFraud* es objetivo

### Distribución de datos

#### Distribución original

In [ ]:
columnas = df_train.iloc[:, :5]

for columna in columnas:
    plt.hist(df[columna])
    plt.title(f"{columna}")
    plt.show()

#### Distribución con **MinMaxScaler**

In [ ]:
# Escalar con MinMaxScaler
scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(scaler_minmax.fit_transform(df_train), columns=df_train.columns)

In [ ]:
# Seleccionar las primeras 5 columnas (ajusta el número si es necesario)
df_minmax_5 = df_minmax.iloc[:, :5]

# Calcular el número de filas y columnas necesarias
n_filas = (len(df_minmax_5.columns) + 4) // 5  # división entera hacia arriba
n_columnas = min(5, len(df_minmax_5.columns))  # máximo de 5 columnas

# Crear subplots
fig, axes = plt.subplots(n_filas, n_columnas, figsize=(15, 5 * n_filas))  # Ajusta el tamaño según necesites

# Distribución de cada columna
flat_axes = axes.ravel()  # Aplanar el arreglo de ejes bidimensional
for i, col in enumerate(df_minmax_5.columns):
    sns.displot(data=df_minmax_5[col], ax=flat_axes[i])
    flat_axes[i].set_title(col)

# Ocultar ejes sobrantes (opcional)
if len(flat_axes) > len(df_minmax_5.columns):
    for ax in flat_axes[len(df_minmax_5.columns):]:
        ax.axis('off')

plt.tight_layout()  # Ajustar espacio entre subplots
plt.show()


#### Distribución con **StandarScaler**

In [ ]:
# Escalar con StandardScaler
scaler_standard = StandardScaler()
df_standard = pd.DataFrame(scaler_standard.fit_transform(df_train), columns=df_train.columns)

In [ ]:
# Seleccionar las primeras 5 columnas (ajusta el número si es necesario)
df_standard_5 = df_standard.iloc[:, :5]

# Calcular el número de filas y columnas necesarias
n_filas = (len(df_standard_5.columns) + 4) // 5  # división entera hacia arriba
n_columnas = min(5, len(df_standard_5.columns))  # máximo de 5 columnas

# Crear subplots
fig, axes = plt.subplots(n_filas, n_columnas, figsize=(15, 5 * n_filas))  # Ajusta el tamaño según necesites

# Distribución de cada columna
flat_axes = axes.ravel()  # Aplanar el arreglo de ejes bidimensional
for i, col in enumerate(df_standard_5.columns):
    sns.displot(data=df_minmax_5[col], ax=flat_axes[i])
    flat_axes[i].set_title(col)

# Ocultar ejes sobrantes (opcional)
if len(flat_axes) > len(df_standard_5.columns):
    for ax in flat_axes[len(df_standard_5.columns):]:
        ax.axis('off')

plt.tight_layout()  # Ajustar espacio entre subplots
plt.show()


#### Entonces el df a ocupar es **minmax scaler**

## Etapa de entrenamiento 🤖

### Preparo data

In [ ]:
df_train =  df_minmax

In [ ]:
df_train.head(1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train, var_obj, test_size=0.2, random_state = 21)

### Preparo modelo **LogisticRegression con Sklearn**

In [ ]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

### Puntaje del modelo

In [ ]:
score_train = modelo.score(X_train, y_train)
print('logistic regresion Pipeline score -> ', score_train)

### Validación del modelo

In [ ]:
score_valid = modelo.score(X_valid, y_valid)
print('logistic regresion val score -> ', score_valid)

### Margen de error y precisión

In [ ]:
predicciones = modelo.predict(X_valid)
precision = accuracy_score(y_valid, predicciones)

print(f"Precisión -> {precision:.2f} \n")

print('Error cuadratico medio del modelo -> ', mean_squared_error(predicciones, y_valid, squared=False))

### Puntaje de resultados

In [ ]:
y_results = modelo.predict(df_test)
score_test = modelo.score(df_test, y_results)
print('logistic regresion test results score -> ', score_valid)

### Exporto data de resultados

In [ ]:
res = pd.DataFrame({'id': df_sub.id,
                    'IsFraud': y_results})
res.to_csv('submission.csv',index=False)